In [3]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv('../results/result.csv')
df = df[df.run >= 3].copy()
df['speed'] = df['sim-cycles'] / df['time']
df['ipc'] = df['instructions'] / df['cycles']
df['MPKI'] = df['cache-misses'] / df['instructions'] * 1000
df

,simulator,benchmark,sim-cycles,run,time,all_data_cache_accesses,cache-misses,cycles,instructions,speed,ipc,MPKI
3,circt-verilator,RocketCore,100000,3,229967,538312851.0,29882.0,500219879.0,1.207403e+09,0.434845,2.413744,0.024749
4,circt-verilator,RocketCore,100000,4,233358,542559903.0,31168.0,507140518.0,1.207403e+09,0.428526,2.380805,0.025814
5,circt-verilator,RocketCore,100000,5,231270,539072010.0,30945.0,501752023.0,1.207403e+09,0.432395,2.406373,0.025629
6,circt-verilator,RocketCore,100000,6,236979,542544132.0,36082.0,513936214.0,1.207403e+09,0.421978,2.349324,0.029884
7,circt-verilator,RocketCore,100000,7,230389,539072307.0,31291.0,500741660.0,1.207403e+09,0.434049,2.411229,0.025916
...,...,...,...,...,...,...,...,...,...,...,...,...
385,ksim,RocketCore,100000,5,150517,449439243.0,19709.0,330581676.0,1.020769e+09,0.664377,3.087797,0.019308
386,ksim,RocketCore,100000,6,150682,449344190.0,22951.0,330957563.0,1.020769e+09,0.663649,3.084290,0.022484
387,ksim,RocketCore,100000,7,148185,449205737.0,19333.0,325063417.0,1.020769e+09,0.674832,3.140215,0.018940
388,ksim,RocketCore,100000,8,148026,449266609.0,19265.0,325212750.0,1.020769e+09,0.675557,3.138773,0.018873


# Speedup

In [5]:
piv = df.pivot_table(values='speed', index='benchmark', columns='simulator')
order = [
  'SHA256',
  'StreamComp',
  'FMUL',
  'FPU',
  'Gemmini',
  'SIGMA',
  'GEMM',
  'Conv2D',
  'RISCVMini',
  'RocketTile',
  'SodorTile'
]
sim_order = [
  'circt-verilator', 'verilator', 'ksim', 'essent', 'repcut-2', 'vcs'
]
baseline_name = 'circt-verilator'
baseline = piv[baseline_name].copy()
for col in piv:
  piv[col] /= baseline
piv = piv.stack().loc[order].reset_index(name='speedup')
piv = piv.set_index('benchmark').reset_index()
def mark_npos(bar, y, *args, **kws):
  npos = []
  for c in bar.containers:
    for child in c.get_children():
      if np.isnan(child.get_height()):
        center = child.get_x() + child.get_width() / 2
        npos.append(center)
  xlim = plt.xlim()
  plt.plot(npos, np.full(len(npos), y), *args, **kws)
  plt.xlim(xlim)

NameError: name 'baseline_name' is not defined

In [ ]:
plt.figure(figsize=(22, 2.5))
bar = sns.barplot(
  data=piv,
  x='benchmark', y='speedup',
  hue='simulator', hue_order=sim_order,
  palette='muted',
  width=0.7,
)
# set_hatch(bar)
# for c in bar.containers:
#   bar.bar_label(c, fmt='$%.1f$', fontsize=10)
mark_npos(bar, 0.3, 'rx', markersize=10, label='compiler crash')
handles, labels = plt.gca().get_legend_handles_labels()
plt.legend(handles[1:] + handles[:1], labels[1:] + labels[:1], loc='upper right', ncol=3)
# plt.xticks(fontsize=13, rotation=-15, ha='left')
plt.xticks(fontsize=13)
plt.xlabel(None)
plt.ylim([0, 5])
plt.ylabel('Speedup', fontsize=14)
ax = plt.gca()
ax.yaxis.set_major_locator(ticker.IndexLocator(base=1, offset=0))
ax.yaxis.set_minor_locator(ticker.IndexLocator(base=0.5, offset=0))
plt.grid(axis='y', alpha=0.3)
plt.grid(axis='y', which='minor', alpha=0.3)
sns.despine()
plt.subplots_adjust(left=0.05, right=0.97, bottom=0.2)
plt.draw()

In [6]:
piv_abs = df.pivot_table(values='speed', columns='simulator', index='benchmark', aggfunc='mean')
baseline_name = 'circt-verilator'
baseline = piv_abs[baseline_name].copy()
piv = piv_abs.apply(lambda x: x / baseline)
piv.loc['Average'] = piv.mean()
styled = piv.style
styled.highlight_max(color='green', axis=1).format('{:.2f}')

simulator,circt-verilator,ksim,verilator
benchmark,,,
Conv2D,1.00,1.73,0.97
FMUL,1.00,2.61,0.73
FPU,1.00,2.71,0.56
GEMM,1.00,2.15,0.74
Gemmini,1.00,4.47,1.40
RISCVMini,1.00,1.80,0.80
RocketCore,1.00,1.56,0.37
RocketTile,1.00,0.92,0.73
SHA256,1.00,2.18,0.67


# IPC

In [168]:
piv = df.pivot_table(values='ipc', columns='simulator', index='benchmark', aggfunc='mean')
piv.loc['Average'] = piv.mean()
styled = piv.style
styled.highlight_max(color='green', axis=1).format('{:.3f}')

simulator,circt-verilator,essent,ksim,repcut-2,vcs,verilator
benchmark,,,,,,
Conv2D,1.801,1.130,2.839,1.540,2.562,1.862
FMUL,1.894,0.689,2.993,nan,1.338,2.091
FPU,3.630,2.600,3.755,1.664,2.489,2.741
GEMM,1.743,1.865,2.904,1.413,2.582,1.441
Gemmini,2.921,0.527,3.289,0.525,2.281,2.683
RISCVMini,2.003,2.522,3.525,1.118,2.528,2.278
RocketCore,2.364,1.274,3.098,0.807,2.350,2.409
RocketTile,2.519,nan,2.965,nan,nan,2.212
SHA256,3.233,3.441,4.298,0.495,2.522,2.789


# Cycles

In [169]:
def format_scled(c):
  units = ['', ' k', ' M', ' G', ' T', ' E']
  for unit in units:
    if c < 1000:
      return f'{c:.1f}{unit}'
    c /= 1000

In [170]:
piv = df.pivot_table(values='cycles', columns='simulator', index='benchmark', aggfunc='mean')
# piv.loc['Average'] = piv.mean()
styled = piv.style
styled.highlight_min(color='green', axis=1).format(format_scled)

simulator,circt-verilator,essent,ksim,repcut-2,vcs,verilator
benchmark,,,,,,
Conv2D,874.4 M,584.4 M,540.7 M,113.0 M,909.9 M,91.0 M
FMUL,2.5 G,2.3 G,952.4 M,None,2.7 G,343.4 M
FPU,451.9 M,46.3 M,168.4 M,78.5 M,695.6 M,85.8 M
GEMM,649.2 M,252.0 M,309.1 M,87.8 M,912.0 M,90.4 M
Gemmini,12.7 G,2.9 G,2.8 G,7.1 G,10.7 G,905.7 M
RISCVMini,190.8 M,48.3 M,107.4 M,57.1 M,661.3 M,24.9 M
RocketCore,510.9 M,261.8 M,329.5 M,444.7 M,921.9 M,139.3 M
RocketTile,412.5 M,None,456.6 M,None,None,59.4 M
SHA256,33.7 M,5.7 M,17.7 M,21.1 M,626.0 M,6.4 M


# Instructions

In [171]:
def format_scled(c):
  units = ['', ' k', ' M', ' G', ' T', ' E']
  for unit in units:
    if c < 1000:
      return f'{c:.1f}{unit}'
    c /= 1000

In [173]:
piv = df.pivot_table(values='instructions', columns='simulator', index='benchmark', aggfunc='mean')
piv.loc['Average'] = piv.mean()
styled = piv.style
styled.highlight_min(color='green', axis=1).format(format_scled)

simulator,circt-verilator,essent,ksim,repcut-2,vcs,verilator
benchmark,,,,,,
Conv2D,1.6 G,660.3 M,1.5 G,174.1 M,2.3 G,169.4 M
FMUL,4.7 G,1.6 G,2.9 G,None,3.6 G,718.2 M
FPU,1.6 G,120.2 M,632.4 M,130.6 M,1.7 G,235.2 M
GEMM,1.1 G,469.7 M,897.3 M,124.1 M,2.4 G,130.2 M
Gemmini,37.2 G,1.5 G,9.3 G,3.7 G,24.4 G,2.4 G
RISCVMini,382.0 M,121.8 M,378.5 M,50.6 M,1.7 G,56.7 M
RocketCore,1.2 G,333.5 M,1.0 G,358.0 M,2.2 G,335.5 M
RocketTile,1.0 G,None,1.4 G,None,None,131.3 M
SHA256,108.1 M,19.7 M,76.0 M,10.4 M,1.6 G,17.8 M


# MPKI

In [174]:
piv = df.pivot_table(values='MPKI', columns='simulator', index='benchmark', aggfunc='mean')
piv.loc['Average'] = piv.mean()
styled = piv.style
styled.highlight_min(color='green', axis=1).format('{:.3f}')

simulator,circt-verilator,essent,ksim,repcut-2,vcs,verilator
benchmark,,,,,,
Conv2D,0.033,6.534,0.029,5.857,1.692,0.189
FMUL,0.011,37.718,0.010,nan,18.301,0.089
FPU,0.019,0.164,0.030,2.848,1.085,0.126
GEMM,0.028,1.124,0.022,12.636,0.965,0.250
Gemmini,14.960,69.927,8.768,31.745,5.268,8.152
RISCVMini,0.083,0.159,0.052,6.665,1.100,0.481
RocketCore,0.033,0.070,0.020,1.685,1.556,0.096
RocketTile,0.031,nan,0.016,nan,nan,0.237
SHA256,0.279,0.938,0.237,19.439,1.236,1.470
